In [ ]:
import pandas as pd
import pyarrow.parquet as pq
from google.cloud import bigquery
import pyarrow
import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/lravin667/Downloads/decisive-scion-437014-c8-d5fd4b94b067.json'
from google.cloud import storage

def read_parquet_from_gcs(bucket_name, parquet_file_path):
    # Initialize a GCS client
    client = storage.Client()

    # Get the bucket
    bucket = client.get_bucket(bucket_name)

    # Create a blob object from the file path
    blob = bucket.blob(parquet_file_path)

    # Download the blob to a local file
    local_file_name = 'temp.parquet'  # Temporary file to store the downloaded Parquet file
    blob.download_to_filename(local_file_name)

    # Read the Parquet file into a pandas DataFrame
    df = pd.read_parquet(local_file_name)

    return df
    
def load_parquet_to_partitioned_bigquery(dataset_id, table_id, source_uri, partition_column):
    # Initialize the BigQuery client
    client = bigquery.Client()

    # Create a job configuration for loading the Parquet file
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.PARQUET,
        # Specify partitioning
        time_partitioning=bigquery.TimePartitioning(
            field=partition_column,  # The column to partition by
            type_=bigquery.TimePartitioningType.DAY,  # Partitioning type (DAY, MONTH, YEAR)
        ),
    )

    # Load the data into the partitioned BigQuery table
    load_job = client.load_table_from_uri(
        source_uri,
        f'{client.project}.{dataset_id}.{table_id}',
        job_config=job_config,
    )

    # Wait for the job to complete
    load_job.result()

    # Output the result
    print(f'Loaded {load_job.output_rows} rows into {dataset_id}.{table_id}.')

if __name__ == '__main__':
        # Define your bucket and file path
    bucket_name = 'lakshmi_test1'  # Replace with your bucket name
    parquet_file_path = 'yellow_tripdata_2024-01.parquet'  # Replace with your Parquet file path

    # Read the Parquet file and print the DataFrame
    df_jan = read_parquet_from_gcs(bucket_name, parquet_file_path)
    parquet_file_path = 'yellow_tripdata_2024-02.parquet'
    df_feb = read_parquet_from_gcs(bucket_name, parquet_file_path)
    inner_join = pd.merge(df_jan, df_jan, on='VendorID', how='inner')
    print('Data from inner join')
    print(inner_join)
    
    # Define your dataset and table
    dataset_id = 'yellow_tripdata'  # dataset ID
    table_id = 'yellow_tripdata_2024-01'       # table ID
    source_uri = 'gs://lakshmi_test1/yellow_tripdata_2024-01.parquet'  # GCS path where the file is located
    partition_column='VendorID'  #partition column on which table will be partitioned

    load_parquet_to_bigquery(dataset_id, table_id, source_uri)
